In [1]:
import numpy as np
from IPython.display import clear_output, display

from pydrake.all import (
    AbstractValue, AddMultibodyPlantSceneGraph, 
    DiagramBuilder, JacobianWrtVariable, JointSliders, 
    LeafSystem, MeshcatVisualizer, Parser, RigidTransform, 
    RollPitchYaw, StartMeshcat
)

from manipulation import FindResource, running_as_notebook
from manipulation.scenarios import AddPackagePaths

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [ ]:
class PrintJacobian(LeafSystem): 
    
    def __init__(self, plant, frame): 
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._frame = frame
        self.DeclareVectorInputPort('state', plant.num_multibody_states())
        self.DeclareForcedPublishEvent(self.Publish)
        
    def Publish(self, context): 
        state = self.get_input_port().Eval(context)
        self._plant.SetPositionsAndVelocities(self._plant_context, state)
        W = self._plant.world_frame()
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, 
            JacobianWrtVariable.kQDot, 
            self._frame, 
            [0, 0, 0], 
            W, 
            W
        )
        
        print("J_G: ")
        print(np.array2string(J_G, formatter={
            'float': lambda x: "{:5.1f}".format(x)
        }))
        print(
            f"smallest singular value(J_G) : {np.min(np.linalg.svd(J_G, compute_uv=False))}"
        )
        clear_output(wait=True)
    
def jacobian_example(): 
    builder = DiagramBuilder()
        
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
    parser = Parser(plant)
    AddPackagePaths(parser)
    parser.AddAllModelsFromFile(
        FindResource("models/iiwa_and_wsg.dmd.yaml")
    )
    plant.Finalize()
        
    meshcat.Delete()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph.get_query_output_port(), meshcat
    )
      
    G = plant.GetBodyByName("body").body_frame()
    print_jacobian = builder.AddSystem(PrintJacobian(plant, G))
    builder.Connect(plant.get_state_output_port(), 
                  print_jacobian.get_input_port())
        
    default_interactive_timeout = None if running_as_notebook else 1.0
    sliders = builder.AddSystem(JointSliders(meshcat, plant))
    diagram = builder.Build()
    sliders.Run(diagram, default_interactive_timeout)
    meshcat.DeleteAddedControls()

meshcat.DeleteAddedControls()
jacobian_example()

J_G: 
[[  0.0   1.0   0.0  -0.2   0.7   0.5   0.8   0.0   0.0]
 [  0.0   0.0  -0.1  -1.0  -0.2   0.8  -0.4   0.0   0.0]
 [  1.0   0.0   1.0  -0.1   0.7  -0.3   0.4   0.0   0.0]
 [  0.2   0.0   0.1  -0.3   0.0   0.0  -0.0   0.0   0.0]
 [  0.4  -0.7   0.4   0.0   0.0  -0.1  -0.0   0.0   0.0]
 [  0.0  -0.2   0.0   0.4  -0.0  -0.1   0.0   0.0   0.0]]
smallest singular value(J_G) : 0.027012349630897308
